<a href="https://colab.research.google.com/github/machinehistories/3d_topography_generator_notebook/blob/main/mesh_from_place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 🌍 SRTM to 3D Mesh Generator (Colab, fixed)
# Avoids make errors by downloading tiles directly
# ==========================================

!pip install geopy rasterio numpy trimesh pyproj shapely elevation requests tqdm

from google.colab import drive
drive.mount('/content/drive')

from google.colab import output
output.enable_custom_widget_manager()

import os
import numpy as np
import rasterio
from rasterio.mask import mask
from geopy.geocoders import Nominatim
from shapely.geometry import box, mapping
from pyproj import Geod, Transformer
import trimesh
import requests
from tqdm import tqdm
import zipfile

# --- Step 1: User Inputs ---
PLACE_NAME = "Mount Shasta, California"
SIZE_METERS = 10000
OUTPUT_FORMAT = "obj"
SAVE_DIR = "/content/drive/MyDrive/SRTM_Meshes"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Step 2: Geocode the place name ---
geolocator = Nominatim(user_agent="srtm_to_mesh_colab")
location = geolocator.geocode(PLACE_NAME)
if not location:
    raise ValueError(f"Could not find location '{PLACE_NAME}'")
lat, lon = location.latitude, location.longitude
print(f"📍 Found location: {PLACE_NAME} at ({lat:.5f}, {lon:.5f})")

# --- Step 3: Compute bounding box in degrees ---
geod = Geod(ellps="WGS84")
lon_min, _, _ = geod.fwd(lon, lat, -90, SIZE_METERS / 2)
lon_max, _, _ = geod.fwd(lon, lat, 90, SIZE_METERS / 2)
_, lat_min, _ = geod.fwd(lon, lat, 180, SIZE_METERS / 2)
_, lat_max, _ = geod.fwd(lon, lat, 0, SIZE_METERS / 2)
bounds = (min(lon_min, lon_max), min(lat_min, lat_max), max(lon_min, lon_max), max(lat_min, lat_max))
print(f"🧭 Bounding Box: {bounds}")

# --- Step 4: Download SRTM data directly from NASA (AWS) ---
# Use 1 arc-second data (~30m). We’ll compute the tile name automatically.
tile_lat = int(np.floor(lat))
tile_lon = int(np.floor(lon))
ns = "N" if tile_lat >= 0 else "S"
ew = "E" if tile_lon >= 0 else "W"
tile_name = f"{ns}{abs(tile_lat):02d}{ew}{abs(tile_lon):03d}"
url = f"https://s3.amazonaws.com/elevation-tiles-prod/skadi/{ns}{abs(tile_lat):02d}/{tile_name}.hgt.gz"

dem_gz = f"/content/{tile_name}.hgt.gz"
dem_hgt = dem_gz.replace(".gz", "")

# Download if not already cached
if not os.path.exists(dem_hgt):
    print(f"⬇️ Downloading {tile_name} from {url}")
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        raise ValueError("Could not download SRTM tile (tile may not exist for this region).")
    with open(dem_gz, "wb") as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk)
    import gzip, shutil
    with gzip.open(dem_gz, "rb") as f_in, open(dem_hgt, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    print("✅ Decompressed DEM")

# --- Step 5: Read and crop the DEM ---
with rasterio.open(dem_hgt, driver="SRTMHGT") as src:
    bbox_geom = box(*bounds)
    out_image, out_transform = mask(src, [mapping(bbox_geom)], crop=True)
    data = out_image[0]
    transform = out_transform

print(f"✅ DEM cropped: shape={data.shape}")

# --- Step 6: Convert DEM to mesh safely ---

ny, nx = data.shape
data = np.where(data < -1000, np.nan, data)  # mark invalid data

xs = np.arange(nx)
ys = np.arange(ny)
xv, yv = np.meshgrid(xs, ys)
lon_grid, lat_grid = rasterio.transform.xy(transform, yv, xv)
lon_grid = np.array(lon_grid)
lat_grid = np.array(lat_grid)

# Convert lat/lon to meters
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x_m, y_m = transformer.transform(lon_grid, lat_grid)

# Flatten arrays
X = x_m.flatten()
Y = y_m.flatten()
Z = data.flatten()

# Create vertices (keeping full grid)
vertices = np.column_stack((X, Y, np.nan_to_num(Z, nan=0)))

# Build faces safely (skip NaNs)
faces = []
cols = nx
rows = ny
for j in range(rows - 1):
    for i in range(cols - 1):
        idx = j * cols + i
        # Indices of the 4 corners of the cell
        v1, v2, v3, v4 = idx, idx + 1, idx + cols, idx + cols + 1
        # Skip if any vertex has NaN elevation
        if np.isnan(Z[v1]) or np.isnan(Z[v2]) or np.isnan(Z[v3]) or np.isnan(Z[v4]):
            continue
        faces.append([v1, v2, v3])
        faces.append([v2, v4, v3])

faces = np.array(faces, dtype=np.int32)

mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
print(f"✅ Mesh generated with {len(vertices)} vertices and {len(faces)} faces")

# --- Step 7: Export Mesh ---
filename_base = PLACE_NAME.replace(",", "").replace(" ", "_")
output_path = os.path.join(SAVE_DIR, f"{filename_base}.{OUTPUT_FORMAT}")
mesh.export(output_path)
print(f"💾 Saved mesh to: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📍 Found location: Mount Shasta, California at (41.40920, -122.19486)
🧭 Bounding Box: (-122.25466402336349, 41.364177553776486, -122.1350649766365, 41.45421749276164)
✅ DEM cropped: shape=(325, 432)
✅ Mesh generated with 140400 vertices and 277134 faces
💾 Saved mesh to: /content/drive/MyDrive/SRTM_Meshes/Mount_Shasta_California.obj


In [ ]:
# ==========================================
# 🧭 Interactive 3D Viewer (Colab, shaded terrain, float32 safe)
# ==========================================
!pip install pythreejs ipywidgets

import numpy as np
from ipywidgets import interact, FloatSlider
from pythreejs import (
    BufferAttribute, BufferGeometry, Mesh, MeshLambertMaterial,
    AmbientLight, PerspectiveCamera, Scene, Renderer, OrbitControls
)

# --- Use the existing mesh variable ---
verts = mesh.vertices.astype(np.float32)   # <-- ensure float32
faces = mesh.faces.astype(np.uint32)

# --- Center mesh at origin ---
center = verts.mean(axis=0)
verts_centered = verts - center

# --- Compute vertex normals ---
if mesh.vertex_normals is None or len(mesh.vertex_normals) == 0:
    mesh.compute_vertex_normals()
normals = mesh.vertex_normals.astype(np.float32)  # <-- ensure float32

# --- Fake directional light for shading ---
light_dir = np.array([1, -1, 2], dtype=np.float32)
light_dir /= np.linalg.norm(light_dir)

# --- Compute vertex brightness ---
brightness = np.clip(np.dot(normals, light_dir), 0, 1)

# --- Map brightness to colors (float32 0..1) ---
colors = np.zeros_like(verts_centered, dtype=np.float32)
colors[:, 0] = 0.6 * brightness + 0.2  # R
colors[:, 1] = 0.4 * brightness + 0.2  # G
colors[:, 2] = 0.2 * brightness + 0.1  # B

# --- Build geometry ---
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(verts_centered, normalized=False),
        'color': BufferAttribute(colors, normalized=False)
    }
)
geometry.index = BufferAttribute(faces.flatten(), normalized=False)

# --- Material & mesh ---
material = MeshLambertMaterial(vertexColors='VertexColors', side='DoubleSide')
terrain_mesh = Mesh(geometry, material)

# --- Camera setup ---
max_range = np.ptp(verts_centered, axis=0).max()
camera = PerspectiveCamera(
    position=[0, -max_range*2, max_range],
    fov=60,
    up=[0,0,1]
)
camera.lookAt([0,0,0])

# --- Scene & renderer ---
fill_light = AmbientLight(intensity=1.0)
scene = Scene(children=[terrain_mesh, fill_light, camera], background='lightblue')
renderer = Renderer(
    scene=scene,
    camera=camera,
    controls=[OrbitControls(controlling=camera)],
    width=800,
    height=600,
    antialias=True
)
display(renderer)

# --- Height exaggeration slider ---
@interact(Height_Exaggeration=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0))
def exaggerate(Height_Exaggeration):
    scaled_verts = verts_centered.copy()
    scaled_verts[:, 2] = verts_centered[:, 2] * Height_Exaggeration
    geometry.attributes['position'].array = scaled_verts
    geometry.attributes['position'].needsUpdate = True


Renderer(camera=PerspectiveCamera(fov=60.0, position=(0.0, -26716.0, 13358.0), projectionMatrix=(1.0, 0.0, 0.0…

interactive(children=(FloatSlider(value=1.0, description='Height_Exaggeration', max=5.0, min=0.5), Output()), …

In [ ]:
# ==========================================
# 🧭 Interactive 3D Viewer (Colab, visible mesh)
# ==========================================
!pip install pythreejs ipywidgets

from ipywidgets import interact, FloatSlider
from pythreejs import (
    BufferAttribute, BufferGeometry, Mesh, MeshLambertMaterial,
    AmbientLight, PerspectiveCamera, Scene, Renderer, OrbitControls
)
import numpy as np

# --- Prepare mesh data ---
verts = mesh.vertices.astype(np.float64)
faces = mesh.faces.astype(np.uint32)

# --- Center mesh at origin ---
center = verts.mean(axis=0)
verts_centered = verts - center

# --- Build geometry ---
geometry = BufferGeometry(
    attributes={'position': BufferAttribute(verts_centered, normalized=False)}
)
geometry.index = BufferAttribute(faces.flatten(), normalized=False)

# --- Material ---
material = MeshLambertMaterial(color='saddlebrown', side='DoubleSide')

# --- Mesh ---
terrain = Mesh(geometry, material)

# --- Compute bounding box for camera zoom ---
max_range = np.ptp(verts_centered, axis=0).max()

# --- Camera ---
camera = PerspectiveCamera(
    position=[0, -max_range*2, max_range],
    fov=60,
    up=[0,0,1]
)
camera.lookAt([0,0,0])

# --- Scene & light ---
fill_light = AmbientLight(intensity=1.0)
scene = Scene(children=[terrain, fill_light, camera], background='lightblue')

# --- Renderer ---
renderer = Renderer(
    scene=scene,
    camera=camera,
    controls=[OrbitControls(controlling=camera)],
    width=800,
    height=600,
    antialias=True
)
display(renderer)

# --- Height exaggeration slider ---
@interact(Height_Exaggeration=FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0))
def exaggerate(Height_Exaggeration):
    scaled_verts = verts_centered.copy()
    scaled_verts[:, 2] = verts_centered[:, 2] * Height_Exaggeration
    geometry.attributes['position'].array = scaled_verts
    geometry.attributes['position'].needsUpdate = True




Renderer(camera=PerspectiveCamera(fov=60.0, position=(0.0, -26716.497518833727, 13358.248759416863), projectio…

interactive(children=(FloatSlider(value=1.0, description='Height_Exaggeration', max=5.0, min=0.5), Output()), …

In [ ]:
# ==========================================
# 🧭 Interactive 3D Viewer (Colab, using k3d)
# ==========================================
!pip install k3d trimesh

import k3d
import trimesh
import os
import numpy as np # Import numpy

# --- Load the saved mesh ---
PLACE_NAME = "Mount Shasta, California" # Ensure this matches the name used for saving
filename_base = PLACE_NAME.replace(",", "").replace(" ", "_")
SAVE_DIR = "/content/drive/MyDrive/SRTM_Meshes" # Ensure this matches the save directory
output_path = os.path.join(SAVE_DIR, f"{filename_base}.obj")

if not os.path.exists(output_path):
    raise FileNotFoundError(f"Mesh file not found at {output_path}")

mesh = trimesh.load_mesh(output_path)

# --- Create k3d plot ---
plot = k3d.Plot(zoom_to_object=True)

# Add the mesh to the plot
# k3d expects vertices as a flat array, but trimesh already provides this
# Removing color to avoid obscuring the mesh
plot += k3d.mesh(mesh.vertices.tolist(), mesh.faces.tolist())

# Set camera position (adjust as needed)
# plot.camera = [x, y, z, look_at_x, look_at_y, look_at_z, up_x, up_y, up_z]
# plot.camera = [
#     mesh.centroid[0], mesh.centroid[1] - np.ptp(mesh.vertices[:,1])*2, mesh.centroid[2] + np.ptp(mesh.vertices[:,2])*1.5, # Camera position
#     mesh.centroid[0], mesh.centroid[1], mesh.centroid[2], # Look at point
#     0, 0, 1 # Up vector
# ]

# Display the plot
plot.display()

Output()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()